In [39]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding, SpatialDropout1D
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping, ModelCheckpoint
import os

In [40]:
df = pd.read_csv("Train.csv")

In [41]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8401 entries, 0 to 8400
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   ID         1800 non-null   float64
 1   Feed       8397 non-null   object 
 2   Sentiment  8401 non-null   object 
dtypes: float64(1), object(2)
memory usage: 197.0+ KB


In [42]:
df.drop('ID',axis=1,inplace=True)

In [43]:
df['Sentiment']=df['Sentiment'].astype(str)
df['Feed']=df['Feed'].astype(str)

In [23]:
pip install tokenizers

     ---------------------------------------- 3.5/3.5 MB 681.3 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [44]:
texts = df["Feed"].tolist()
sentiments = df["Sentiment"].tolist()

# Initialize tokenizer
tokenizer = Tokenizer()

# Train tokenizer
tokenizer.fit_on_texts(texts)

# Save tokenizer as JSON file
with open("tokenizerV5.json", "w") as f:
    json.dump(tokenizer.to_json(), f)


In [45]:
import json
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pandas as pd

# Load the dataset from csv

# Extract the texts and labels
texts = df['Feed'].values
labels = df['Sentiment'].values

# Initialize a tokenizer with a given number of words to keep
num_words = 10000
tokenizer = Tokenizer(num_words=num_words)

# Fit the tokenizer on the texts
tokenizer.fit_on_texts(texts)

# Convert the texts to sequences of integers using the tokenizer
sequences = tokenizer.texts_to_sequences(texts)

# Pad the sequences to have the same length
max_len = 100
padded_sequences = pad_sequences(sequences, maxlen=max_len)

# Save the tokenizer as JSON
tokenizer_json = tokenizer.to_json()
with open('tokenizerV6.json', 'w', encoding='utf-8') as f:
    f.write(tokenizer_json)

In [28]:
tokenizer = Tokenizer(num_words=5000, lower=True)
tokenizer.fit_on_texts(df['Feed'].values)
X = tokenizer.texts_to_sequences(df['Feed'].values)
X = pad_sequences(X, maxlen=250)

AttributeError: 'Tokenizer' object has no attribute 'to_str'

In [83]:
Y = pd.get_dummies(df['Sentiment']).values

In [84]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)


In [85]:
model = Sequential()
model.add(Embedding(5000, 128, input_length=X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(256, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2, activation='softmax'))

In [86]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [87]:
early_stopping = EarlyStopping(monitor='val_loss', patience=3, verbose=1, mode='min')
checkpoint = ModelCheckpoint('model.h5', monitor='val_loss', save_best_only=True, verbose=1, mode='min')


In [88]:
history = model.fit(X_train, Y_train, epochs=20, batch_size=32, validation_split=0.2, callbacks=[early_stopping, checkpoint])


Epoch 1/20
168/168 [==============================] - ETA: 0s - loss: 0.6322 - accuracy: 0.6927
Epoch 1: val_loss improved from inf to 0.42915, saving model to model.h5
168/168 [==============================] - 360s 2s/step - loss: 0.6322 - accuracy: 0.6927 - val_loss: 0.4292 - val_accuracy: 0.8132
Epoch 2/20
168/168 [==============================] - ETA: 0s - loss: 0.2858 - accuracy: 0.8787
Epoch 2: val_loss improved from 0.42915 to 0.29728, saving model to model.h5
168/168 [==============================] - 373s 2s/step - loss: 0.2858 - accuracy: 0.8787 - val_loss: 0.2973 - val_accuracy: 0.8698
Epoch 3/20
168/168 [==============================] - ETA: 0s - loss: 0.1929 - accuracy: 0.9165
Epoch 3: val_loss improved from 0.29728 to 0.27725, saving model to model.h5
168/168 [==============================] - 351s 2s/step - loss: 0.1929 - accuracy: 0.9165 - val_loss: 0.2773 - val_accuracy: 0.8690
Epoch 4/20
168/168 [==============================] - ETA: 0s - loss: 0.1371 - accuracy: 

In [100]:
score = model.evaluate(X_test, Y_test, verbose=0)
print("Accuracy: %.2f%%" % (score[1]*100))


Accuracy: 87.63%


X_test

In [102]:
Y_test

array([[1, 0],
       [1, 0],
       [0, 1],
       ...,
       [1, 0],
       [1, 0],
       [0, 1]], dtype=uint8)

In [1]:
import pickle
with open('modeldarija.pkl', 'wb') as file:
    pickle.dump(model, file)

NameError: name 'model' is not defined

In [97]:
df['Feed']

0       m3alem bourjilia w illi ma yefehmouch yelzmou ...
1                                             Ya m3alllam
2                           Ma7lek! Zin ou fannena 7loua.
3                          hhhhh ya bliiiiiiiida ya Hanen
4                                Nikraha w ma5yebha pffff
                              ...                        
8396    نوفل ما يجيب كان المنح وخوف ملا زبالة منال عما...
8397    نسبة مشاهدة كبيرة رغم التشويش الى صار ...برافو...
8398                                 Mala ta7ana makyebha
8399                                                Tefah
8400                         walah ka3ba le mamstk pfffff
Name: Feed, Length: 8401, dtype: object

In [121]:
text = "maalem wlh"
text_sequence = tokenizer.texts_to_sequences([text])
text_sequence_padded = pad_sequences(text_sequence, maxlen=250)
prediction = model.predict(text_sequence_padded)[0]
predicted_label = np.argmax(prediction)
print(predicted_label)

1/1 [==============================] - 0s 108ms/step
1


In [2]:
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import json

# Load CSV dataset
df = pd.read_csv('Train.csv')
df.drop("ID",axis=1,inplace=True)
# Preprocess text data
# Remove punctuations and special characters
df['Feed'] = df['Feed'].str.replace('[^a-zA-Z0-9\s]', '')
# Convert text to lowercase
df['Feed'] = df['Feed'].str.lower()
# Replace NaN values with empty strings
df['Feed'].fillna('', inplace=True)

# Tokenize text data
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(df['Feed'].values)

# Save tokenizer as a JSON file
with open('tokenizer.json', 'w', encoding='utf-8') as f:
    f.write(tokenizer.to_json())

C:\Users\ASUS\AppData\Local\Temp\ipykernel_6904\192509584.py:11: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Feed'] = df['Feed'].str.replace('[^a-zA-Z0-9\s]', '')


In [47]:
from tensorflow.keras.models import load_model
model = load_model('modeldarijaV2.h5')

In [50]:
test_texts = ['This is a positive sentence.', 'This is a negative sentence.']
test_sequences = tokenizer.texts_to_sequences(test_texts)
test_padded_sequences = pad_sequences(test_sequences)
predictions = model.predict(test_padded_sequences)

ValueError: in user code:

    File "C:\Users\ASUS\anaconda3\lib\site-packages\keras\engine\training.py", line 2169, in predict_function  *
        return step_function(self, iterator)
    File "C:\Users\ASUS\anaconda3\lib\site-packages\keras\engine\training.py", line 2155, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\ASUS\anaconda3\lib\site-packages\keras\engine\training.py", line 2143, in run_step  **
        outputs = model.predict_step(data)
    File "C:\Users\ASUS\anaconda3\lib\site-packages\keras\engine\training.py", line 2111, in predict_step
        return self(x, training=False)
    File "C:\Users\ASUS\anaconda3\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\ASUS\anaconda3\lib\site-packages\keras\engine\input_spec.py", line 298, in assert_input_compatibility
        raise ValueError(

    ValueError: Input 0 o

In [2]:
from tensorflow.keras.models import load_model

# Load the saved model
model = load_model('modeldarijaV2.h5')

# Display the model summary
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 250, 128)          640000    
                                                                 
 spatial_dropout1d (SpatialD  (None, 250, 128)         0         
 ropout1D)                                                       
                                                                 
 lstm_4 (LSTM)               (None, 256)               394240    
                                                                 
 dense_4 (Dense)             (None, 2)                 514       
                                                                 
Total params: 1,034,754
Trainable params: 1,034,754
Non-trainable params: 0
_________________________________________________________________
